# P3 - Movie Genre Classification - Random Forest

Elaborado por GRUPO 1:

- Juanita Piraban Barbosa - 201216313
- Lorena Morales Rodríguez - 202027957
- Alejandro Barinas Guio - 201628859
- Jaime Humberto Trujillo Perea - 201920366
- Alexander Zapata Galindo - 201425426

### Objective

Classify a movie genre based on its plot.
Source: https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

- Input: movie plot
- Output: Probability of the movie belong to each genre

### Evaluation

- 50% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

### Deatline

- The project must be carried out in the groups assigned.
- Use clear and rigorous procedures.
- The delivery of the project is on August 1st, 2021, 11:59 pm, through Bloque Neón.
- No projects will be received after the delivery time or by any other means than the one established.

### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.
See https://arxiv.org/abs/1702.01992

## Sample Submission

In [57]:
# Librerías

import pandas as pd
from matplotlib import pyplot as plt
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

import nltk
from nltk.corpus import stopwords

In [58]:
df_train = pd.read_csv('DataSet/dataTraining.csv', encoding='UTF-8', index_col=0)
df_test = pd.read_csv('Dataset/dataTesting.csv', encoding='UTF-8', index_col=0)

In [59]:
df_train.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [60]:
df_train['rating'] = df_train['rating'].astype(int)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7895 entries, 3107 to 215
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    7895 non-null   int64 
 1   title   7895 non-null   object
 2   plot    7895 non-null   object
 3   genres  7895 non-null   object
 4   rating  7895 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 370.1+ KB


In [61]:
df_test.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [62]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3383 entries, 1 to 11275
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    3383 non-null   int64 
 1   title   3383 non-null   object
 2   plot    3383 non-null   object
dtypes: int64(1), object(2)
memory usage: 105.7+ KB


### Create y

In [63]:
df_train['genres'] = df_train['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(df_train['genres'])

In [64]:
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

### Create count vectorizer


In [87]:
## eliminar caracteres especiales
X= pd.DataFrame(df_train['plot'])
X1= pd.DataFrame(df_test['plot'])

spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–",'$','®','¡',
             '£','°','¹','½','à','á','â','ä','å','ç',
              'è','é','ê','ë','í','ï','ñ','ò','ó','ô',
              'ö','ø','ù','ú','û','ü','ʼ']
for char in spec_chars:
    X = X.apply(lambda x: x.str.replace(char, ''))
    X1 = X1.apply(lambda x: x.str.replace(char, ''))

X= X['plot']
X1= X1['plot']

#Limpiar el vocabulario
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

X= X.apply(clean_text)
X1= X1.apply(clean_text)

In [88]:
#english_stopwords = stopwords.words('english')

vect = CountVectorizer(max_features=25000 , ngram_range=(1, 1))
X_dtm = vect.fit_transform(X)

In [89]:
print(vect.get_feature_names())

['aa', 'aames', 'aang', 'aaron', 'abalam', 'abalone', 'abandon', 'abandoned', 'abandoning', 'abandons', 'abberline', 'abbey', 'abbie', 'abbott', 'abby', 'abc', 'abdomen', 'abduct', 'abducted', 'abducting', 'abduction', 'abductor', 'abducts', 'abdullah', 'abe', 'abel', 'abernathy', 'aberration', 'abhorrent', 'abhorring', 'abigail', 'abilities', 'ability', 'abin', 'abital', 'abkani', 'able', 'abm', 'abner', 'abnormal', 'abnormality', 'aboard', 'abode', 'abolitionist', 'abomination', 'aborigines', 'abort', 'abortion', 'abortions', 'abound', 'abra', 'abraham', 'abrams', 'abrasive', 'abroad', 'abronsius', 'abrupt', 'abruptly', 'abscond', 'absconded', 'absence', 'absences', 'absent', 'absentee', 'absolute', 'absolutely', 'absolution', 'absorbed', 'abstinence', 'abstract', 'absurd', 'abu', 'abuse', 'abused', 'abuser', 'abuses', 'abusing', 'abusive', 'abyss', 'acacio', 'academic', 'academically', 'academics', 'academy', 'acapulco', 'accent', 'accented', 'accept', 'acceptable', 'acceptance', 'a

### Train / Test DataSets

In [90]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=1234)

In [91]:
X_train

<5289x25000 sparse matrix of type '<class 'numpy.int64'>'
	with 293047 stored elements in Compressed Sparse Row format>

In [92]:
X_test

<2606x25000 sparse matrix of type '<class 'numpy.int64'>'
	with 143574 stored elements in Compressed Sparse Row format>

In [93]:
y_train_genres

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [94]:
y_test_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]])

### Initial multi-class / multi-label model

In [95]:
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=None, random_state=1234))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(n_jobs=-1,
                                                     random_state=1234))

In [96]:
y_pred_genres = clf.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.7946080198080869

### Tuning model

In [97]:
## hiperparameters
#n_estimators = [100]
#random_state = [1234]
#bootstrap = ['True']    
#max_features = ['auto', 'sqrt'] # Number of features to consider at every split
#max_depth = [10, 15, 20, 25] #  Maximum number of levels in tree[2, 5, 10, 15, 'None']
#min_samples_split = [2, 5, 7, 10] # Minimum number of samples required to split a node[2, 3, 4, 5]
#min_samples_leaf = [1, 5, 7, 10] # Minimum number of samples required at each leaf node[1, 2, 3, 4, 5]
#parameters = {
#               'estimator__n_estimators':n_estimators,
#               'estimator__random_state':random_state,
#               'estimator__bootstrap':bootstrap,
#               'estimator__max_features': max_features,
#               'estimator__max_depth': max_depth,
#               'estimator__min_samples_split': min_samples_split,
#               'estimator__min_samples_leaf': min_samples_leaf
#            }
#
## model_to_set
#clf = OneVsRestClassifier(RandomForestClassifier())
#
## model_to_tune
#tuning = GridSearchCV(estimator=clf, param_grid=parameters, cv=10, n_jobs=-1, refit='roc_auc_score')
#tuning.fit(X_train, y_train_genres)

In [98]:
# best parameters
#best_param = tuning.best_params_
#best_param

In [99]:
# best parameters after tuning
best_param_1={'estimator__bootstrap': 'True',
 'estimator__max_depth': 25,
 'estimator__max_features': 'auto',
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 5,
 'estimator__n_estimators': 100,
 'estimator__random_state': 1234}

In [100]:
# best_model
best_model = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, bootstrap=True, random_state=1234,
                                                max_features=best_param_1['estimator__max_features'],
                                                max_depth=best_param_1['estimator__max_depth'],
                                                min_samples_split=best_param_1['estimator__min_samples_split'],
                                                min_samples_leaf=best_param_1['estimator__min_samples_leaf']))
best_model.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=25,
                                                     min_samples_split=5,
                                                     n_jobs=-1,
                                                     random_state=1234))

In [101]:
# ROC_AUC best training model
y_pred_genres = best_model.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro',multi_class='ovr')

0.8078210366110339

### Precit over test dataframe

In [103]:
# Dataframe to predict
df_test.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [104]:
X_test_dtm = vect.fit_transform(X1)
print(vect.get_feature_names())

['aa', 'aaa', 'aadland', 'aardvark', 'aaron', 'aarons', 'aaronson', 'aback', 'abandon', 'abandonded', 'abandoned', 'abandoning', 'abandonment', 'abandons', 'abarra', 'abb', 'abbe', 'abbey', 'abbie', 'abbot', 'abbott', 'abby', 'abdicate', 'abdomen', 'abdomens', 'abduct', 'abducted', 'abducting', 'abduction', 'abductions', 'abductor', 'abductors', 'abducts', 'abdul', 'abdullah', 'abe', 'abel', 'abelhammer', 'abercrombie', 'abetting', 'abhor', 'abhorrent', 'abhors', 'abide', 'abiding', 'abigail', 'abilene', 'abilities', 'ability', 'abin', 'able', 'abm', 'abner', 'aboard', 'abode', 'abol', 'abolish', 'abomination', 'aborted', 'abortion', 'aborts', 'abound', 'abounded', 'abounds', 'abraham', 'abramovich', 'abrams', 'abrasive', 'abroad', 'abrupt', 'abruptly', 'absconded', 'absconds', 'absence', 'absences', 'absent', 'absentee', 'absolom', 'absolute', 'absolutely', 'absolution', 'absorbed', 'absorbing', 'absorbs', 'abstract', 'absurd', 'absurdity', 'absurdly', 'abudhabi', 'abuse', 'abused', '

In [105]:
# Predict over df_test
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred=best_model.predict_proba(X_test_dtm)
y_pred=pd.DataFrame(y_pred, index=df_test.index, columns=cols)
y_pred.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.115923,0.088184,0.018635,0.046522,0.368938,0.125338,0.047993,0.469462,0.061010,0.080229,...,0.053759,0.080778,0.000040,0.175819,0.056430,0.007631,0.050348,0.192269,0.020368,0.014458
4,0.142457,0.112743,0.028892,0.031797,0.362682,0.125726,0.032030,0.459258,0.061010,0.070881,...,0.020585,0.071500,0.000040,0.169598,0.065932,0.020777,0.019117,0.200373,0.048509,0.026680
5,0.254087,0.223823,0.109958,0.053847,0.425005,0.230563,0.200674,0.428405,0.125550,0.215518,...,0.060190,0.131243,0.000040,0.213144,0.123942,0.014629,0.174721,0.257856,0.031993,0.109283
6,0.157019,0.123955,0.028441,0.110163,0.372079,0.166023,0.068794,0.488353,0.122504,0.137146,...,0.062171,0.098414,0.000031,0.188477,0.097291,0.005417,0.031599,0.208792,0.102376,0.047501
7,0.146066,0.116063,0.084651,0.167364,0.378107,0.135349,0.067511,0.460272,0.190002,0.105646,...,0.102734,0.068703,0.000040,0.279548,0.075325,0.005456,0.045235,0.207253,0.038011,0.026903


In [106]:
# Export
y_pred.to_csv('pred_genres_text_RF-210801-1.csv', index_label='ID')